In [ ]:
from hepmc import *
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [ ]:
ndim = 2
nsamples = 1000

target = densities.Camel(ndim)
# add .count attributes to pdf and pot_gradient methods
util.count_calls(target, 'pdf', 'pot_gradient')
start = np.full(ndim, 1/3)

# Markov Updates
The package defines the abstract `MarkovUpdate` which implements a general `sample` function
based on the `next_state(current)` function, that each concrete update must override.
The sample returned is an instance of `MarkovSample` and contains the generated sample as `data`
and provides information such as the mean, variance, and acceptance rate for Metropolis updates.

## Metropolis Updates
Metropolis updates are generally subclasses of `MetropolisUpdate`, which is abstract.
The "default" Metropolis (Hasting) update is implemented in `DefaultMetropolis`.
The metheods, on instantiation, take the sample dimension and a callable target probability
as arguments. The pdf can be a `Density` object, but as opposed to Hamiltonian updates, they
can also be simple functions.

## Default Metropolis Hasting

In [ ]:
# reset counts
target.pdf.count = 0
target.pdf.pot_gradient = 0

# by default the range for Uniform ins [0, 1]^ndim
proposal = densities.Uniform(ndim)
sampler = DefaultMetropolis(ndim, target, proposal)

# sample
t_start = timer()
sample = sampler.sample(nsamples, start, log_every=nsamples/4)
t_end = timer()

# print info
print('time: ', t_end - t_start)
print('pdf calls: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample  # show sample

## Adaptive Metropolis

In [ ]:
nadapt = 1000
nburnin = 1000

# reset counts
target.pdf.count = 0
target.pdf.pot_gradient = 0

# hast to be a proposal with cov attribute
# this is what adaptive metropolis modifies
proposal = densities.Gaussian(ndim, mu=0.5, cov=0.005)
metropolis_sampler = AdaptiveMetropolisUpdate(
    ndim, target.pdf, proposal, t_initial=100,
    adapt_schedule=lambda t: t <= nadapt)

# burn in
metropolis_sampler.sample(nburnin, [0.5, 0.5], log_every=0)
metropolis_sampler.is_adaptive = False  # turn adaptation off

t_start = timer()
sample = metropolis_sampler.sample(nsamples, start, log_every=nsamples/4)
t_end = timer()

sample.target = target

print('time: ', t_end - t_start)
print('pdf calls: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample

## Hamiltonian Updates
Hamiltonian updates take densities as argument, and therefore don't require a dedicated ndim argument.

In [ ]:
step_size = 0.0041
steps = 40

# reset counts
target.pdf.count = 0
target.pdf.pot_gradient = 0

momentum_dist = densities.Gaussian(ndim, scale=1)
sampler = hamiltonian.HamiltonianUpdate(target, momentum_dist, steps, step_size)

t_start = timer()
sample = sampler.sample(nsamples, start, log_every=nsamples/4)
t_end = timer()

print('time: ', t_end - t_start)
print('pdf calss: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample

In [ ]:
ndim = 2
nsamples = 1000

target = densities.Gaussian(ndim, [1/3, 1/3], cov=0.1**2/2)
# add .count attributes to pdf and pot_gradient methods
util.count_calls(target, 'pdf', 'pot_gradient')
start = np.full(ndim, 1/3)

step_size = .0074
steps = 40
    
# reset counts
target.pdf.count = 0
target.pdf.pot_gradient = 0

momentum_dist = densities.Gaussian(ndim, cov=1)
sampler = hamiltonian.HamiltonianUpdate(target, momentum_dist, steps, step_size)

t_start = timer()
sample = sampler.sample(nsamples, [.4, .4], log_every=nsamples/4)
t_end = timer()

print('time: ', t_end - t_start)
print('pdf calss: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
print(util.bin_wise_chi2(sample, 20))
sample

In [ ]:
ndim = 2
nsamples = 1000

target = densities.Camel(2)
# add .count attributes to pdf and pot_gradient methods
util.count_calls(target, 'pdf', 'pot_gradient')
start = np.full(ndim, 1/3)

step_size = .0074
steps = 40
    
# reset counts
target.pdf.count = 0
target.pdf.pot_gradient = 0

beta = 0.7
momentum_dist = densities.Gaussian(ndim, cov=1)
sampler_local = hamiltonian.HamiltonianUpdate(target, momentum_dist, steps, step_size)
sampler_is = DefaultMetropolis(2, target)
sampler = MixingMarkovUpdate(2, [sampler_is, sampler_local], [beta, 1-beta])

t_start = timer()
sample = sampler.sample(nsamples, [.4, .4], log_every=nsamples/4)
t_end = timer()

print('time: ', t_end - t_start)
print('pdf calss: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
print(util.bin_wise_chi2(sample, 20))
sample

In [ ]:
ndim = 2
nsamples = 1000

target = densities.Gaussian(ndim, [1/3, 1/3], cov=0.1**2/2)
# add .count attributes to pdf and pot_gradient methods
util.count_calls(target, 'pdf', 'pot_gradient')
start = np.full(ndim, 1/3.01)

nadapt = 500

def adapt_schedule(t):
    return t <= nadapt

# reset counts
target.pdf.count = 0
target.pdf.pot_gradient = 0

momentum_dist = densities.Gaussian(ndim, scale=50)
sampler = hamiltonian.NUTSUpdate(target, momentum_dist, adapt_schedule)

t_start = timer()
sample = sampler.sample(nsamples, start, log_every=nsamples/4)
t_end = timer()

print('time: ', t_end - t_start)
print('pdf calss: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample

In [ ]:
sampler.step_size_min

## Dual Averaging HMC

In [ ]:
nadapt = 1000

def adapt_schedule(t):
    return t <= nadapt

# reset counts
target.pdf.count = 0
target.pdf.pot_gradient = 0

momentum_dist = densities.Gaussian(ndim, scale=.05)
sampler = hamiltonian.DualAveragingHMC(target, momentum_dist, 1, adapt_schedule)

t_start = timer()
sample = sampler.sample(nsamples, start, log_every=nsamples/4)
t_end = timer()

print('time: ', t_end - t_start)
print('pdf calss: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample 

## No-U-turn sampler

In [ ]:
nadapt = -1

def adapt_schedule(t):
    return t <= nadapt

target = densities.Gaussian(ndim, [1/3, 1/3], cov=0.1**2/2)
util.count_calls(target, 'pdf', 'pot_gradient')
# reset counts
target.pdf.count = 0
target.pot_gradient.count = 0

momentum_dist = densities.Gaussian(ndim, scale=50)
sampler = hamiltonian.NUTSUpdate(target, momentum_dist, adapt_schedule)

t_start = timer()
sample = sampler.sample(nsamples, start, log_every=nsamples/4)
t_end = timer()

print('time: ', t_end - t_start)
print('pdf calss: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample

## Spherical HMC
Sampling is by implementation restrained to $[0, 1]^{ndim}$.

In [ ]:
# reset counts
target.pdf.count = 0
target.pdf.pot_gradient = 0

sampler = hamiltonian.StaticSphericalHMC(target, 0.01, 1, 1, 15)

t_start = timer()
sample = sampler.sample(nsamples, start, log_every=nsamples/4)
t_end = timer()

print('time: ', t_end - t_start)
print('pdf calss: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample

## Spherical NUTS

In [ ]:
nadapt = 100

def adapt_schedule(t):
    return t <= nadapt

# reset counts
target.pdf.count = 0
target.pdf.pot_gradient = 0

sampler = hamiltonian.SphericalNUTS(target, adapt_schedule)

t_start = timer()
sample = sampler.sample(nsamples, start, log_every=nsamples/4)
t_end = timer()

print('time: ', t_end - t_start)
print('pdf calss: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample

# Importance Sampling - Metropolis

In [ ]:
nopt = 1000  # integration steps

# imperfect channels
channels = MultiChannel([
    densities.Gaussian(ndim, mu=1/5, cov=0.005),
    densities.Gaussian(ndim, mu=4/5, cov=0.005)])

# multi channel integrator
importance = MultiChannelMC(channels)

t_start = timer()
integration_sample = importance(target, [], [nopt], []) # integrate

sampler = DefaultMetropolis(ndim, target.pdf, channels)
sample = sampler.sample(nsamples, start, log_every=nsamples/4)
t_end = timer()

sample

## MC 3
### Uniform local sampler

In [ ]:
dist = [densities.Gaussian(ndim, mu=np.random.rand(2), scale=0.2) for i in range(10)]
sampler = mc3.MC3Uniform(target, MultiChannel(dist), 0.1, beta=.8)
sample = sampler(([], [1000], []), nsamples)
sample.target = target
sample

### HMC local sampler

In [ ]:
# reset counts
target.pdf.count = 0
target.pdf.pot_gradient = 0

step_size = .0041
steps = 40

dist = [densities.Gaussian(ndim, mu=np.random.rand(ndim), scale=0.2) for i in range(10)]
sampler = mc3.MC3Hamilton(target, MultiChannel(dist), 1., steps, step_size, beta=.2)
sample = sampler(([], [1000], []), 1000, log_every=1000/4)
sample.target = target

print('pdf calss: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample

In [ ]:
util.bin_wise_chi2(sample, 15)

### Trigonometric basis surrogate

In [ ]:
steps = 10
step_size = 0.1
mass = 10
beta = 0
nodes = 1000

# reset counts
target_s = densities.Camel(ndim)

# importance
channels = MultiChannel([densities.Gaussian(ndim, mu=np.random.rand(2), scale=0.2) for i in range(10)])
mc_importance = MultiChannelMC(channels)
integration_sample = mc_importance(target_s, [], [10000], [])

# surrogate
basis = surrogate.extreme_learning.TrigBasis(ndim)
log_vals = -np.ma.log(integration_sample.function_values)
xvals = integration_sample.data[~log_vals.mask]
log_vals = log_vals[~log_vals.mask]

params = basis.extreme_learning_train(xvals, log_vals, nodes)
def surr(xs):
    return basis.eval_gradient(*params, xs)
target_s.pot_gradient = surr


# sampling
p_dist = densities.Gaussian(channels.ndim, scale=mass)
sample_hmc = hamiltonian.HamiltonianUpdate(target_s, p_dist, steps, step_size)

sample_is = DefaultMetropolis(ndim, target_s.pdf, channels)

updates = [sample_is, sample_hmc]
mixing_sampler = MixingMarkovUpdate(ndim, updates, [beta, 1-beta])


sample = mixing_sampler.sample(nsamples, [0.3, 0.4], log_every=nsamples/4)
sample.target = target

print('pdf calls: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample

In [ ]:
# plot 
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100) 
mgrid = np.meshgrid(x, y)

# fit
out = basis.eval_gradient_split(*params, *mgrid)

plt.figure(figsize=(12,8))
plt.subplot(221)
plt.title("surrogate dx")
plt.imshow(out[:,:,0])
plt.colorbar()

plt.subplot(222)
plt.title("surrogate dy")
plt.imshow(out[:,:,1])
plt.colorbar()

grad = target.pot_gradient(np.stack((mgrid[0].flatten(), mgrid[1].flatten()), axis=0).transpose()).reshape((*mgrid[0].shape, 2))
plt.subplot(223)
plt.title("real dx")
plt.imshow(grad[:,:,0])
plt.colorbar()

plt.subplot(224)
plt.title("real dy")
plt.imshow(grad[:,:,1])
plt.colorbar()

plt.tight_layout()
plt.show()

### Gaussian radial basis surrogate

In [ ]:
steps = 30
step_size = 0.1
mass = 10
beta = 0.1
nodes = 1000

# reset counts
target_s = densities.Camel(ndim)

# importance
channels = MultiChannel([densities.Gaussian(ndim, mu=np.random.rand(2), scale=0.2) for i in range(10)])
mc_importance = MultiChannelMC(channels)
integration_sample = mc_importance(target_s, [], [10000], [])

# surrogate
basis = surrogate.extreme_learning.GaussianBasis(ndim)
log_vals = -np.ma.log(integration_sample.function_values)
xvals = integration_sample.data[~log_vals.mask]
log_vals = log_vals[~log_vals.mask]

params = basis.extreme_learning_train(xvals, log_vals, nodes)
def surr(xs):
    return basis.eval_gradient(*params, xs)
target_s.pot_gradient = surr


# sampling
p_dist = densities.Gaussian(channels.ndim, scale=mass)
sample_hmc = hamiltonian.HamiltonianUpdate(target_s, p_dist, steps, step_size)

sample_is = DefaultMetropolis(ndim, target_s, channels)

updates = [sample_is, sample_hmc]
mixing_sampler = MixingMarkovUpdate(ndim, updates, [beta, 1-beta])


sample = mixing_sampler.sample(nsamples, [0.3, 0.4], log_every=nsamples/4)
sample.target = target

print('pdf calls: ', target.pdf.count)
print('pot_gradient calls: ', target.pot_gradient.count)
sample

In [ ]:
# plot 
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100) 
mgrid = np.meshgrid(x, y)

# fit
out = basis.eval_gradient_split(*params, *mgrid)

plt.figure(figsize=(12,8))
plt.subplot(221)
plt.title("surrogate dx")
plt.imshow(out[:,:,0], vmin=-120, vmax=120)
plt.colorbar()

plt.subplot(222)
plt.title("surrogate dy")
plt.imshow(out[:,:,1], vmin=-120, vmax=120)
plt.colorbar()

grad = target.pot_gradient(np.stack((mgrid[0].flatten(), mgrid[1].flatten()), axis=0).transpose()).reshape((*mgrid[0].shape, 2))
plt.subplot(223)
plt.title("real dx")
plt.imshow(grad[:,:,0])
plt.colorbar()

plt.subplot(224)
plt.title("real dy")
plt.imshow(grad[:,:,1])
plt.colorbar()

plt.tight_layout()
plt.show()